In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf

import pandas as pd
import numpy as np
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Downloads data

In [3]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")

In [4]:
s = time.time()
# data = get_data(get_all_tickers())
data = get_data(["BA", "TSLA", "AAPL"])
data_cop = data.copy()
print(time.time() - s)

[*********************100%***********************]  3 of 3 completed
0.5462014675140381


### First DataFrame that contains necessary information

In [18]:
def parse_df(df, old_high=2, high_by="High"):
    # old_high- How many days ago it broke high (x or less)
    # high_by- by which OHLC to determine d
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close, "strategy": 0})

    return new_df

### Filtering out stocks that don't match 

In [19]:
parsed = parse_df(data_cop)
parsed

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$,strategy
AAPL,178.61,198.23,9.90,51418700,2023-07-19,2023-07-19,"9,183,894,038.38",0
BA,223.41,243.10,8.10,12346500,2023-08-01,2023-08-01,"2,758,331,610.21",0
TSLA,238.59,313.80,23.97,106345900,2022-09-21,2022-09-21,"25,373,067,891.55",0


In [17]:
parsed.loc["BA"]

Close                            223.41
52wk High                        243.10
% to 52wk                          8.10
Volume                         12346500
Date of high        2023-08-01 00:00:00
Old date of high    2023-08-01 00:00:00
Volume$                2,758,331,610.21
Name: BA, dtype: object

In [ ]:
def buy_signal